In [1]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


In [1]:
/// This tutorial demostrates the definition, resp. the import of 
/// data for boundary and initial values. 
/// In order to demonstrate the usage, 
/// we employ the exemplaric Poisson solver.

In [1]:
using BoSSS.Application.SipPoisson;

In [1]:
/// We use a temporary database for this tutorial:

In [1]:
var tempDb = CreateTempDatabase();

In [1]:
/// We use the following helper function to create a template for 
/// the multiple solver runs.

In [1]:
Func<SipControl> PreDefinedControl = delegate() {
    SipControl c = new SipControl();
 
    c.SetDGdegree(2);
 
    c.GridFunc = delegate() {
        // define a grid of 10x10 cells
        double[] nodes = GenericBlas.Linspace(-1, 1, 11);
        var grd = Grid2D.Cartesian2DGrid(nodes, nodes);
 
        // set the entire boundary to Dirichlet b.c.
        grd.DefineEdgeTags(delegate (double[] X) {
            return BoundaryType.Dirichlet.ToString();
        });
 
        return grd;
    };
 
    c.SetDatabase(tempDb);
    c.savetodb = true; 
 
    return c;    
};

In [1]:
/// Again, we are using the workflow management

In [1]:
BoSSSshell.WorkflowMgm.Init("Demo_BoundaryAndInitialData");

In [1]:
/// % ======================================
/// \section{Textual and Embedded formulas}
/// % ======================================

In [1]:
SipControl c1 = PreDefinedControl();

In [1]:
/// Provide initial data as a text:

In [1]:
c1.AddInitialValue("RHS","X => Math.Sin(X[0])*Math.Cos(X[1])",
                        TimeDependent:false);

In [1]:
/// Finally, all initial data is stored in the 
/// \code{AppControl.InitialValues} dictionary and 
/// all boundary data is stored in the 
/// \code{AppControl.BoundaryValues} dictionary.
 
/// The common interface for all varinats to specify boundary
/// and initial data is \code{IBoundaryAndInitialData}.
/// The snippet above is only a shortcut to add a \code{Formula} object,
/// which implements the \code{IBoundaryAndInitialData} interface.

In [1]:
c1.InitialValues

In [1]:
c1.InitialValues["RHS"]

In [1]:
/// In \BoSSSpad, such objects can also be extracted from 
/// static methods of classes; note that these should not depend on any other
/// object in the worksheet.

In [1]:
Formula BndyFormula = new Formula(
    "BndyValue.BndyFunction",
    false,
"static class BndyValue {"+
"    public static double BndyFunction(double[] X) {"+
"        return 1.0;"+
"    }"+
"}");

In [1]:
c1.AddBoundaryValue(BoundaryType.Dirichlet.ToString(),
                    "T",
                    BndyFormula);

In [1]:
var J1 = c1.RunBatch();

In [1]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(3600*4);

In [1]:
/// What happened to the job?
J1.Status

In [1]:
/// BoSSScmdSilent
NUnit.Framework.Assert.IsTrue(J1.Status == JobStatus.FinishedSuccessful);

In [1]:
/// % ==================
/// \section{1D Splines}
/// % ==================

In [1]:
/// Splines can be used to interpolate nodal data onto a DG field;
/// currently, only 1D is supported.

In [1]:
SipControl c2 = PreDefinedControl();

In [1]:
// create test data for the spline
double[] xNodes = GenericBlas.Linspace(-2,2,13);
double[] yNodes = xNodes.Select(x => x*0.4).ToArray();

In [1]:
var rhsSpline = new Spline1D(xNodes, yNodes,
                             0,
                             Spline1D.OutOfBoundsBehave.Extrapolate);

In [1]:
/// BoSSScmdSilent
double err = 0;
// test the spline: a line must be interpolated exactly.
foreach(double xtst in GenericBlas.Linspace(-3,3,77)) { 
   double sVal = rhsSpline.Evaluate(new double[] {xtst , 0, 0 }, 0.0);
   double rVal = xtst*0.4;
   err += Math.Abs(sVal - rVal);
}
NUnit.Framework.Assert.Less(err, 1.0e-10, "Slpine implementation fail.");

In [1]:
c2.AddInitialValue("RHS", rhsSpline);

In [1]:
var J2 = c2.RunBatch();

In [1]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(3600*4);

In [1]:
J2.Status

In [1]:
/// BoSSScmdSilent
NUnit.Framework.Assert.IsTrue(J2.Status == JobStatus.FinishedSuccessful);

In [1]:
/// % =====================================================
/// \section{Interpolating values from other Calculations}
/// % =====================================================

In [1]:
/// For demonstrational purposes, we use the result (i.e. the last time-step) 
/// of a previous calculation as a right-hand-side for the next calculation.

In [1]:
var j2Sess = J2.LatestSession;

In [1]:
j2Sess

In [1]:
j2Sess.Timesteps

In [1]:
var lastTimeStep = j2Sess.Timesteps.Last();

In [1]:
/// We encapsulate the value T in the \code{ForeignGridValue} object,
/// which allows interpolation between different meshes:

In [1]:
var newForeignMesh = new ForeignGridValue(lastTimeStep,"T");

In [1]:
/// Use different mesh in the control file:

In [1]:
SipControl c3 = PreDefinedControl();

In [1]:
c3.GridFunc = delegate() {
   // define a grid of *triangle* cells
   double[] nodes = GenericBlas.Linspace(-1, 1, 11);
   var grd = Grid2D.UnstructuredTriangleGrid(nodes, nodes);
 
   // set the entire boundary to Dirichlet b.c.
   grd.DefineEdgeTags(delegate (double[] X) {
       return BoundaryType.Dirichlet.ToString();
   });
 
   return grd;
};
// we also save the RHS in the database
c3.AddFieldOption("RHS", SaveOpt: FieldOpts.SaveToDBOpt.TRUE);

In [1]:
/// finally, we define the RHS:

In [1]:
c3.AddInitialValue("RHS", newForeignMesh);

In [1]:
/// BoSSScmdSilent
double orgProbe = newForeignMesh.Evaluate(new double[] {0.5,0.5}, 0.0);
double newProbe = lastTimeStep.GetField("T").ProbeAt(new double[] {0.5,0.5});
NUnit.Framework.Assert.Less(Math.Abs(orgProbe - newProbe), 1.0e-10, "Check (1) on ForeignGridValue failed");

In [1]:
var J3 = c3.RunBatch();

In [1]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(3600*4);

In [1]:
J3.Status

In [1]:
/// BoSSScmdSilent
NUnit.Framework.Assert.IsTrue(J3.Status == JobStatus.FinishedSuccessful);

In [1]:
/// \paragraph{Side Note:} Since the quadrilateral mesh used for the original
/// right-hand-side is geometrically embedded in the triangular mesh 
/// the interpolation error should be zero (up to machine precision).

In [1]:
var firstTimeStep = J3.LatestSession.Timesteps.First();

In [1]:
DGField RhsOnTriangles = firstTimeStep.GetField("rhs"); // case-insensitive!
DGField RhsOriginal    = lastTimeStep.GetField("T");

In [1]:
// note: we have to cast DGField to ConventionalDGField in order to use
// the 'L2Distance' function:
((ConventionalDGField)RhsOnTriangles).L2Distance((ConventionalDGField)RhsOriginal);

In [1]:
/// BoSSScmdSilent
var H1err = ((ConventionalDGField)RhsOnTriangles).H1Distance((ConventionalDGField)RhsOriginal);
NUnit.Framework.Assert.Less(H1err, 1.0e-10, "Check (2) on ForeignGridValue failed.");

In [1]:
/// % ===================================
/// \section{Restart from Dummy-Sessions}
/// % ===================================

In [1]:
/// Dummy sessions are kind of fake siolver runs, with the only purpose 
/// of using them for a restart.

In [1]:
DGField RHSforRestart = firstTimeStep.GetField("RHS");

In [1]:
/// We save the DG field \code{RHSforRestart} in the database;
/// This automatically creates a timestep and a session which host the DG field:

In [1]:
var RestartTimestep = tempDb.SaveTimestep(RHSforRestart);

In [1]:
RestartTimestep

In [1]:
RestartTimestep.Session

In [1]:
/// This time step can be used as a restart value.:

In [1]:
var c4 = PreDefinedControl();

In [1]:
c4.InitialValues.Clear();
c4.SetRestart(RestartTimestep);

In [1]:
var J4 = c4.RunBatch();

In [1]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(3600*4);

In [1]:
J4.Status

In [1]:
/// BoSSScmdSilent
NUnit.Framework.Assert.IsTrue(J4.Status == JobStatus.FinishedSuccessful);

In [1]:
/// \paragraph{Note:}
/// Since no mesh interpolation is performed for the restart, it is much faster
/// than \code{ForeignGridValue}, but less flexible 
/// (a restart is always performed on the same mesh).
///
/// To avoid multiple mesh interpolations (e.g. when multiple runs are required)
/// one coudl therefore speed up the process by doing the 
/// mesh interpolation once (use \emph{ProjectFromForeignGrid}) in BoSSSpad and
/// save the interpolation in a dummy session.